In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from skimage.filters import threshold_otsu
from sklearn.mixture import GaussianMixture
from scipy.stats import chi2_contingency, fisher_exact

from pathlib import Path
import json
import os

In [ ]:
base_dir = Path().resolve()
base_dir

## Patient Measurement Analysis

In [ ]:
patient_1991 = base_dir.parent / "data/measurements_1991.xlsx"
patinet_3149 = base_dir.parent / "data/measurements_3149.csv"

In [ ]:
# patient 1991
raw = pd.read_excel(patient_1991, header=None)
header = raw.iloc[0,0].split(";")
raw.columns = header
raw["Cell: ALDH1L1: Median"] = raw["Cell: ALDH1L1: Median"].astype(str).str.extract(r"([-+]?\d*\.?\d+)")[0]
raw["Cell: ALDH1L1: Median"] = pd.to_numeric(raw["Cell: ALDH1L1: Median"], errors="coerce")
raw = raw.iloc[3:,:]
df_1991 = raw.reset_index(drop=True)
df_1991

In [ ]:
# patient_3149
raw = pd.read_csv(patinet_3149, header=None)   
header = raw.iloc[0, 0].split(";")
data = raw.iloc[1:, 0].str.split(";", expand=True)
data.columns = header
df_3149 = data.reset_index(drop=True)
df_3149

## EDA on datasets

In [ ]:
data = df_3149 #  df_3149 or df_1991

In [ ]:
for col in data.columns:
    if "Mean" in col or "Median" in col:
        data[col] = pd.to_numeric(data[col], errors="coerce")

In [ ]:
data.columns

### Apply Thresholding Methods on Markers

In [ ]:
def compute_thresholds(data, markers, 
                       methods=["otsu", "gmm", "percentile", "zscore"],
                       percentile=95, 
                       zscore=2, 
                       manual_thr=None, 
                       save_plots=True, 
                       outdir="thresholds"):
    """
    Compute thresholds for multiple markers with different methods.
    For each marker, thresholds will be computed for both Cell and Nucleus (if columns exist).
    Optionally save histograms with thresholds overlaid.

    Parameters
    ----------
    data : pd.DataFrame
        QuPath measurement table.
    markers : list
        List of marker names.
    methods : list
        Methods to use: ["otsu", "gmm", "percentile", "zscore", "manual"]
    percentile : int
        Percentile cutoff for 'percentile' method.
    zscore : float
        Z-score cutoff for 'zscore' method.
    manual_thr : float or dict, optional
        - If float → same manual threshold for all markers.
        - If dict {marker: value} → marker-specific threshold.
        - If dict {marker: {compartment: value}} → marker+compartment-specific.
    save_plots : bool
        Whether to save histograms.
    outdir : str
        Directory to save plots.

    Returns
    -------
    thresholds_df : pd.DataFrame
        DataFrame with thresholds for each marker × compartment × method.
    TH : dict
        Dictionary {marker: {compartment: {method: threshold}}}
    """
    
    if save_plots:
        os.makedirs(outdir, exist_ok=True)

    results = []
    TH = {}
    
    for marker in markers:
        TH[marker] = {}
        
        for compartment in ["Cell", "Nucleus"]:
            col = f"{compartment}: {marker}: Mean"
            if col not in data.columns:
                continue
            
            values = data[col].dropna().values.reshape(-1, 1)
            if len(values) == 0:
                continue

            thr_dict = {"Marker": marker, "Compartment": compartment}
            TH[marker][compartment] = {}

            for method in methods:
                if method == "otsu":
                    thr = threshold_otsu(values)

                elif method == "gmm":
                    gmm = GaussianMixture(n_components=2, random_state=42)
                    gmm.fit(values)
                    means = np.sort(gmm.means_.flatten())
                    thr = np.mean(means)

                elif method == "percentile":
                    thr = np.percentile(values, percentile)

                elif method == "zscore":
                    mean = values.mean()
                    std = values.std()
                    thr = mean + zscore * std

                elif method == "manual":
                    
                    if manual_thr is None:
                        continue
                    elif isinstance(manual_thr, (int, float)):
                        thr = manual_thr
                    elif isinstance(manual_thr, dict):
                        if marker in manual_thr:
                            if isinstance(manual_thr[marker], dict):
                                thr = manual_thr[marker].get(compartment, None)
                                if thr is None:
                                    continue
                            else:
                                thr = manual_thr[marker]
                        else:
                            continue
                    else:
                        continue

                else:
                    continue

                thr_dict[method] = thr
                TH[marker][compartment][method] = thr

                # ---------------- plot & save ----------------
                if save_plots:
                    plt.figure(figsize=(6,4))
                    plt.hist(values, bins=100, alpha=0.7, color="gray")
                    plt.axvline(thr, color="red", linestyle="--", label=f"{method} = {thr:.2f}")
                    plt.title(f"{marker} ({compartment}) - {method}")
                    plt.xlabel("Intensity")
                    plt.ylabel("Cell count")
                    plt.legend()
                    fname = f"{marker}_{compartment}_{method}.png".replace(" ", "_")
                    plt.savefig(os.path.join(outdir, fname), dpi=150, bbox_inches="tight")
                    plt.close()

            results.append(thr_dict)

    thresholds_df = pd.DataFrame(results).set_index(["Marker", "Compartment"])
    return thresholds_df, TH


In [ ]:
markers = ["SPP1", "TMEM119", "CD68", "CD45", "LGALS3", "H3K27M", "GLUT1", "CD31"]

# Manual thresholds 1991:
#  "SPP1": 0.49, # otsu
# "TMEM119": 30, # manual
# "CD68": 9.8, # otsu
# "CD45": 27.8, # otsu
# "LGALS3": 5.76, # otsu
# "H3K27M": 40, # manual
# "GLUT1": 10 , # manual
# "CD31": 6.02 # otsu

# Manual thresholds 3149:
#  "SPP1": 0.52, # otsu
# "TMEM119": 33, # manual
# "CD68": 14.84, # otsu
# "CD45": 38, # manual
# "LGALS3": 4.25 , # otsu
# "H3K27M": 50 , # manual
# "GLUT1": 25 , # manual
# "CD31": 6.48 # manual


thresholds_df, TH = compute_thresholds(data, 
                                       markers,
                                       methods=["otsu", "gmm", "percentile", "zscore","manual"],
                                       percentile=90,
                                       zscore=2,
                                       manual_thr={
                                        "SPP1": 0.52, # otsu
                                        "TMEM119": 33, # manual
                                        "CD68": 14.84, # otsu
                                        "CD45": 38, # manual
                                        "LGALS3": 4.25 , # otsu
                                        "H3K27M": 50 , # manual
                                        "GLUT1": 25 , # manual
                                        "CD31": 6.48 # manual
                                       },
                                       save_plots=True
                                       )

thresholds_df

In [ ]:
def pos(data, marker, method="otsu", compartment="Cell"):
    """
    Return boolean mask of positive cells for a given marker.
    
    Parameters
    ----------
    data : pd.DataFrame
        QuPath measurement table.
    marker : str
        Marker name, e.g. "CD68"
    method : str
        Thresholding method key, e.g. "otsu", "gmm"
    compartment : str
        "Cell" or "Nucleus"
    """
    col = f"{compartment}: {marker}: Mean"
    thr = TH[marker][compartment][method]
    return data[col] > thr

In [ ]:
# Groups
method = "manual"

tmem119 = pos(data, "TMEM119", method, "Cell") & \
                 pos(data, "CD68", method, "Cell") & \
                 pos(data, "CD45", method, "Cell")

lgals3 = pos(data, "LGALS3", method, "Cell") & \
                  pos(data, "CD68", method, "Cell") & \
                  pos(data, "CD45", method, "Cell")

h3k27m = pos(data, "H3K27M", method, "Nucleus")

hypoxic = pos(data, "GLUT1", method, "Cell") & (~pos(data, "CD31", method, "Cell"))

# Results dictionary
results = {}

results["SPP1 in TMEM119+ CD68+ CD45+"] = (
    pos(data.loc[tmem119], "SPP1", method, "Cell").mean() * 100
)

results["SPP1 in TMEM119+ CD68+ CD45+ GLUT1+ CD31-"] = (
    pos(data.loc[tmem119 & hypoxic], "SPP1", method, "Cell").mean() * 100
)

results["SPP1 in LGALS3+ CD68+ CD45+"] = (
    pos(data.loc[lgals3], "SPP1", method, "Cell").mean() * 100
)

results["SPP1 in LGALS3+ CD68+ CD45+ GLUT1+ CD31-"] = (
    pos(data.loc[lgals3 & hypoxic], "SPP1", method, "Cell").mean() * 100
)

results["SPP1 in H3K27M+"] = (
    pos(data.loc[h3k27m], "SPP1", method, "Cell").mean() * 100
)

results["SPP1 in H3K27M+ GLUT1+ CD31-"] = (
    pos(data.loc[h3k27m & hypoxic], "SPP1", method, "Cell").mean() * 100
)


# Summary DataFrame
summary = pd.DataFrame.from_dict(results, orient="index", columns=["% SPP1+"])
summary["% SPP1+"] = summary["% SPP1+"].fillna("No cells")
summary.reset_index(inplace=True, names=["Group"])
summary

In [ ]:
groups = [
    "TMEM119+ CD68+ CD45+", 
    "TMEM119+ CD68+ CD45+ GLUT1+ CD31-",
    "LGALS3+ CD68+ CD45+", 
    "LGALS3+ CD68+ CD45+ GLUT1+ CD31-",
    "H3K27M+", 
    "H3K27M+ GLUT1+ CD31-"
]
values = summary["% SPP1+"]

pairs = [(0,1), (2,3), (4,5)]

plt.figure(figsize=(10,6))

for i, (a, b) in enumerate(pairs):
    x = np.array([i, i+0.35])   
    y = [values[a], values[b]]
    
    plt.bar(x[0], y[0], width=0.3, color="steelblue", label="Baseline" if i == 0 else "")
    plt.bar(x[1], y[1], width=0.3, color="lightcoral", label="Baseline with GLUT1+ CD31-" if i == 0 else "")

    plt.text(x[0], y[0] + 0.5, f"{y[0]:.1f}", ha='center', va='bottom', fontsize=10)
    plt.text(x[1], y[1] + 0.5, f"{y[1]:.1f}", ha='center', va='bottom', fontsize=10)


plt.xticks([i+0.15 for i in range(len(pairs))],
           ["TMEM119+ CD68+ CD45+", "LGALS3+ CD68+ CD45+", "H3K27M+"],
           rotation=0)

plt.ylabel("Percentage of SPP1+ (%)")
plt.title("SPP1+ percentages across cell populations and their GLUT1+ CD31- subsets")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def test_significance(data, group_mask, marker="SPP1", method="otsu", compartment="Cell"):
    """
    Compare %SPP1+ between baseline and hypoxic subsets.
    
    Parameters
    ----------
    data : pd.DataFrame
        QuPath measurement table
    group_mask : dict
        Dictionary with masks, e.g. {"microglia": mask1, "microglia_hypoxic": mask2}
    marker : str
        Marker to test positivity for (default="SPP1")
    method : str
        Thresholding method key, e.g. "otsu"
    compartment : str
        "Cell" or "Nucleus"
    
    Returns
    -------
    results : pd.DataFrame
        Table with counts, percentages and p-values
    """
    results = []
    col = f"{compartment}: {marker}: Mean"
    
    for base_name in set(k.split("_hypoxic")[0] for k in group_mask.keys()):
        mask_base = group_mask.get(base_name, None)
        mask_hyp = group_mask.get(base_name + "_hypoxic", None)
        if mask_base is None or mask_hyp is None:
            continue

        # baseline
        base_vals = data.loc[mask_base, col]
        thr = TH[marker][compartment][method]
        base_pos = (base_vals > thr).sum()
        base_neg = (base_vals <= thr).sum()

        # hypoxic subset
        hyp_vals = data.loc[mask_hyp, col]
        hyp_pos = (hyp_vals > thr).sum()
        hyp_neg = (hyp_vals <= thr).sum()

        # contingency table
        table = np.array([[base_pos, base_neg],
                          [hyp_pos, hyp_neg]])

        # chi-square or Fisher depending on counts
        if (table < 5).any():
            oddsratio, pval = fisher_exact(table)
            test = "Fisher"
        else:
            chi2, pval, _, _ = chi2_contingency(table)
            test = "Chi-square"

        results.append({
            "Group": base_name,
            "Baseline %SPP1+": base_pos / (base_pos + base_neg) * 100 if (base_pos+base_neg) > 0 else np.nan,
            "Hypoxic %SPP1+": hyp_pos / (hyp_pos + hyp_neg) * 100 if (hyp_pos+hyp_neg) > 0 else np.nan,
            "Test": test,
            "p-value": pval
        })

    return pd.DataFrame(results)


In [ ]:
group_mask = {
    "tmem119": tmem119,
    "tmem119_hypoxic": tmem119 & hypoxic,
    "lgals3": lgals3,
    "lgals3_hypoxic": lgals3 & hypoxic,
    "h3k27m": h3k27m,
    "h3k27m_hypoxic": h3k27m & hypoxic
}

summary_stats = test_significance(data, group_mask, marker="SPP1", method="manual", compartment="Cell")
summary_stats


In [ ]:
# info = {
#     "Group": ["TMEM119+ CD68+ CD45+", "TMEM119+ CD68+ CD45+ GLUT1+ CD31-", 
#               "LGALS3+ CD68+ CD45+", "LGALS3+ CD68+ CD45+ GLUT1+ CD31-",
#               "H3K27M+", "H3K27M+ GLUT1+ CD31-"],
#     "Condition": ["Baseline", "GLUT1+ CD31-", "Baseline", "GLUT1+ CD31-", "Baseline", "GLUT1+ CD31-"],
#     "SPP1_percent_patient1991": [71.39,72.35,76.35,73.26,21.42,30.72],
#     "SPP1_percent_patient3149": [42,38,35,25,6.9,4.1]
# }

# df = pd.DataFrame(info)
# df

In [ ]:
# df_long = df.melt(
#     id_vars=["Group", "Condition"],
#     value_vars=["SPP1_percent_patient1991", "SPP1_percent_patient3149"],
#     var_name="Patient", value_name="SPP1_percent"
# )

# df_long["Patient"] = df_long["Patient"].replace({
#     "SPP1_percent_patient1991": "Patient 1991",
#     "SPP1_percent_patient3149": "Patient 3149"
# })

# plt.figure(figsize=(12,6))
# ax = sns.barplot(
#     data=df_long,
#     x="Group", y="SPP1_percent",
#     hue="Patient", dodge=True,
#     palette=["steelblue","lightcoral"]
# )

# for p in ax.patches:
#     ax.annotate(f"{p.get_height():.1f}",
#                 (p.get_x() + p.get_width()/2., p.get_height()),
#                 ha='center', va='bottom', fontsize=9, color="black", xytext=(0,2),
#                 textcoords='offset points')

# plt.xticks(rotation=25, ha="right")
# plt.ylabel("SPP1+ (%)")
# plt.title("SPP1+ percentages across groups (Patient 1991 vs Patient 3149)")
# plt.legend(title="Patient")
# plt.tight_layout()
# plt.show()


# quPath Json Data

In [ ]:
summary_path = base_dir.parent / "qupath/1991_Diffuse midline glioma H3K27M/data/1/summary.json"
with open(summary_path, "r") as f:
    summary = json.load(f)
summary

In [ ]:
server_path = base_dir.parent / "qupath/1991_Diffuse midline glioma H3K27M/data/1/server.json"
with open(server_path, "r") as f:
    server = json.load(f)
server

In [ ]:
classes_path = base_dir.parent / "qupath/1991_Diffuse midline glioma H3K27M/classifiers/classes.json"
with open(classes_path, "r") as f:
    classes = json.load(f)
classes